In [1]:
from numpy import array
from pprint import pprint
from time import time
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
from sklearn import tree
from sklearn import naive_bayes
from sklearn import svm, metrics, preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection  import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import joblib
import pickle
import warnings
warnings.filterwarnings('ignore')
from sklearn import svm
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
df_raw = pd.read_csv('transactions_export.csv', delimiter=';', header=None, names=['Text', 'Category'])
df_raw.head(10)

,Text,Category
0,DIRK VDBROEK FIL4200,Boodschappen
1,AH5501 Amsterdam AMSTERDAM NLD,Boodschappen
2,J Jurg,Augustinus/Kartel
3,Jumbo Leiden Station LEIDEN NLD,Boodschappen
4,St. Expl. Eigen Huis LEIDEN NLD,Augustinus/Kartel
5,Aldi 040 Leiden LEIDEN NLD,Boodschappen
6,Albert Heijn 1565 OEGSTGEEST NLD,Boodschappen
7,bol.com b.v.,Winkels
8,HOOGVLIET 500 05 LEIDEN NLD,Boodschappen
9,Aldi 040 Leiden LEIDEN NLD,Boodschappen


In [3]:
def preprocess_text(text):
    # Remove punctuations and numbers
    text = re.sub('[^a-zA-Z]', ' ', text)

    # Single character removal
    text = re.sub(r'\b\w{1,1}\b', ' ', text)

    # Removing multiple spaces
    text = re.sub(r'\s+', ' ', text)

    return text.lower()

df = df_raw
df['Text'] = df_raw['Text'].apply(preprocess_text)
df.head(10)

,Text,Category
0,dirk vdbroek fil,Boodschappen
1,ah amsterdam amsterdam nld,Boodschappen
2,jurg,Augustinus/Kartel
3,jumbo leiden station leiden nld,Boodschappen
4,st expl eigen huis leiden nld,Augustinus/Kartel
5,aldi leiden leiden nld,Boodschappen
6,albert heijn oegstgeest nld,Boodschappen
7,bol com,Winkels
8,hoogvliet leiden nld,Boodschappen
9,aldi leiden leiden nld,Boodschappen


In [4]:
X = df['Text']
le = preprocessing.LabelEncoder()
y = le.fit_transform(df['Category'].tolist())

In [5]:
algorithms = {
    "Linear Support Vector Classifier": svm.LinearSVC(),
    "SGD Classifier": SGDClassifier(),
    "Random Forest Classifier": RandomForestClassifier(),
    "Multinomial Naive Bayes": naive_bayes.MultinomialNB(),
    "Bernoulli Naive Bayes":naive_bayes.BernoulliNB(),
    "K-nearest Neighbors": KNeighborsClassifier(n_neighbors=5),
    "Decision Tree": tree.DecisionTreeClassifier()
}

scores = []
failures = []

for name, sk_class in algorithms.items():
    text_clf = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', sk_class),
    ])
    start = time()
    score = cross_val_score(text_clf, X, y, cv=5)
    end = time()
    time_elapsed = str(round(end - start, 4)) + "s"
    scores.append([score.mean()*100, name, time_elapsed])
    print("{: >40} {: >15}".format(name, time_elapsed))

print("\n\nResults:")
for score in sorted(scores, key=lambda x: 1 - x[0]):
    print("{: >40} & {: >15}\% & {: >15} \\\ ".format(score[1], round(score[0], 2), score[2]))
    
print("\nFailures:")
for x in failures:
    print(x)

        Linear Support Vector Classifier         0.1801s
                          SGD Classifier         0.1143s
                Random Forest Classifier         0.1706s
                 Multinomial Naive Bayes         0.0672s
                   Bernoulli Naive Bayes          0.056s
                     K-nearest Neighbors         0.1596s
                           Decision Tree         0.0686s


Results:
        Linear Support Vector Classifier &           97.67\% &         0.1801s \\ 
                          SGD Classifier &           97.67\% &         0.1143s \\ 
                Random Forest Classifier &           96.98\% &         0.1706s \\ 
                           Decision Tree &           96.35\% &         0.0686s \\ 
                     K-nearest Neighbors &           95.05\% &         0.1596s \\ 
                 Multinomial Naive Bayes &           92.86\% &         0.0672s \\ 
                   Bernoulli Naive Bayes &           85.48\% &          0.056s \\ 

Failures

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)          

pipeline = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', svm.LinearSVC()),
])
text_clf = pipeline
text_clf.fit(X_train, y_train)
svm_predictions = text_clf.predict(X_test)

print(metrics.accuracy_score(y_test, svm_predictions)*100)
target_names = le.inverse_transform(np.unique(np.sort(y_test).tolist()))
print(metrics.classification_report(y_test, svm_predictions, target_names=target_names))

99.375
                   precision    recall  f1-score   support

Augustinus/Kartel       0.94      1.00      0.97        15
     Boodschappen       1.00      1.00      1.00       115
             Eten       0.98      1.00      0.99        54
          Kleding       1.00      0.88      0.93         8
             Loon       1.00      1.00      1.00         5
     Nacht winkel       1.00      1.00      1.00         3
               OV       1.00      1.00      1.00        14
           Pinnen       1.00      1.00      1.00         1
    Spaarrekening       1.00      1.00      1.00        23
         Telefoon       1.00      1.00      1.00         3
Uitgaan/festivals       1.00      0.88      0.93         8
   Vaste cashflow       1.00      1.00      1.00        47
         Vrienden       1.00      1.00      1.00        16
          Winkels       1.00      1.00      1.00         8

         accuracy                           0.99       320
        macro avg       0.99      0.98      0.9

In [7]:
predictions = le.inverse_transform(svm_predictions)
actual = le.inverse_transform(y_test)

for idx, input, prediction, label in zip(enumerate(X_test), X_test, predictions, actual):
    if prediction != label:
        print("No.", idx[0], 'input,',input, ', has been classified as', prediction, 'and should be', label)

No. 223 input, studentendiscotheek leiden nld , has been classified as Augustinus/Kartel and should be Uitgaan/festivals
No. 270 input, primark netherlands  , has been classified as Eten and should be Kleding


In [8]:
def predict_label(transaction_text):
    input_text = preprocess_text(transaction_text)
    prediction = text_clf.predict([input_text])
    predicted_label = le.inverse_transform([prediction])
    print(predicted_label[0])
    
predict_label("augustinus")

Augustinus/Kartel


In [9]:
joblib.dump(predict_label, '../model.pkl', protocol=2)

['../model.pkl']